In [161]:
!pip install --upgrade pip
!pip install ibm-watson-machine-learning gradio pandas numpy python-dotenv ibm-cloud-sdk-core
!pip install ibm_watson_machine_learning==1.0.367
!pip install python-dotenv==1.1.0
!pip install faker==37.1.0
!pip install openpyxl==3.1.5
!pip install  huggingface_hub==0.20.2
!pip install sentence-transformers==2.2.0
!pip uninstall huggingface_hub transformers sentence-transformers -y
!pip install sentence-transformers --upgrade
!pip install langchain
!pip install ibm_watsonx_ai
!pip install python-dotenv==1.1.0
!pip install faker==37.1.0
!pip install openpyxl==3.1.5
!pip uninstall huggingface_hub transformers sentence-transformers -y
!pip install sentence-transformers --upgrade
!pip install torch --upgrade --index-url https://download.pytorch.org/whl/cpu
!pip install faiss-cpu
!pip install -U langchain-community


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

In [41]:
!pip show huggingface_hub
!pip show transformers
!pip show torch

Name: huggingface-hub
Version: 0.30.2
Summary: Client library to download and publish models, datasets and other repos on the huggingface.co hub
Home-page: https://github.com/huggingface/huggingface_hub
Author: Hugging Face, Inc.
Author-email: julien@huggingface.co
License: Apache
Location: /root/venv/lib/python3.10/site-packages
Requires: filelock, fsspec, packaging, pyyaml, requests, tqdm, typing-extensions
Required-by: gradio, gradio_client, sentence-transformers, tokenizers, transformers
Name: transformers
Version: 4.51.3
Summary: State-of-the-art Machine Learning for JAX, PyTorch and TensorFlow
Home-page: https://github.com/huggingface/transformers
Author: The Hugging Face team (past and future) with the help of all our contributors (https://github.com/huggingface/transformers/graphs/contributors)
Author-email: transformers@huggingface.co
License: Apache 2.0 License
Location: /root/venv/lib/python3.10/site-packages
Requires: filelock, huggingface-hub, numpy, packaging, pyyaml, reg

Connecting credentials

In [4]:
import os
os.environ["WATSONX_API_KEY"] = ""
os.environ["WATSONX_PROJECT_ID"] = ""



In [7]:
from ibm_watson_machine_learning import APIClient

wml_credentials = {
    "apikey": os.getenv("WATSONX_API_KEY"),
    "url": "https://us-south.ml.cloud.ibm.com"  # Verify correct region
}

client = APIClient(wml_credentials)
client.set.default_project(os.getenv("WATSONX_PROJECT_ID"))


'SUCCESS'

In [10]:
# Check your .env file contents
import os
from dotenv import load_dotenv

load_dotenv()
print("API Key exists:", bool(os.getenv("WATSONX_API_KEY")))  # Should return True
print("Project ID exists:", bool(os.getenv("WATSONX_PROJECT_ID")))  # Should return True


API Key exists: True
Project ID exists: True


In [13]:
import os
from dotenv import load_dotenv
from ibm_watson_machine_learning.foundation_models import Model

# Load environment variables
load_dotenv()

# Initialize Watsonx client
credentials = {
    "apikey": os.getenv("WATSONX_API_KEY"),
    "url": "https://us-south.ml.cloud.ibm.com"
}

model_id = "ibm/granite-13b-instruct-v2"
project_id = os.getenv("WATSONX_PROJECT_ID")

model = Model(
    model_id=model_id,
    credentials=credentials,
    project_id=project_id
)


# Buliding Rag architecture

Preparing the Rag corpus

In [16]:
from faker import Faker
import random
from datetime import datetime, timedelta
import pandas as pd

fake = Faker()

# Configuration
NUM_MANAGERS = 3
NUM_EMPLOYEES = 10
NUM_TASKS = 10
NUM_FEEDBACKS = 15
NUM_COMMENTS = 20
NUM_SKILLS = 5
MAX_SKILLS_PER_USER = 3

# 1. Generate Users (managers + employees)
users = []
for i in range(NUM_MANAGERS):
    users.append({
        "UserID": i + 1,
        "FirstName": fake.first_name(),
        "LastName": fake.last_name(),
        "IsActive": True,
        "Role": "manager",
        "EmployeeNumber": f"MGR{i+1:03d}",
        "ReportsToUserID": None,
        "TeamID": f"TEAM{i+1}",
        "TeamName": f"Team {fake.color_name()}"
    })

for i in range(NUM_EMPLOYEES):
    manager = random.choice(users[:NUM_MANAGERS])
    users.append({
        "UserID": NUM_MANAGERS + i + 1,
        "FirstName": fake.first_name(),
        "LastName": fake.last_name(),
        "IsActive": True,
        "Role": "employee",
        "EmployeeNumber": f"EMP{i+1:03d}",
        "ReportsToUserID": manager["UserID"],
        "TeamID": manager["TeamID"],
        "TeamName": manager["TeamName"]
    })

# 2. Generate Skills
skills = []
for i in range(NUM_SKILLS):
    skills.append({
        "SkillID": i + 1,
        "SkillName": fake.job(),
        "Description": fake.sentence()
    })

# 3. Generate UserSkills (many-to-many)
user_skills = []
for user in users:
    skill_ids = random.sample(range(1, NUM_SKILLS + 1), random.randint(1, MAX_SKILLS_PER_USER))
    for skill_id in skill_ids:
        user_skills.append({
            "UserID": user["UserID"],
            "SkillID": skill_id
        })

# 4. Generate Goals and Tasks
goals = []
tasks = []
for i, manager in enumerate(users[:NUM_MANAGERS]):
    goal_id = i + 1
    goals.append({
        "GoalID": goal_id,
        "GoalName": f"Goal {goal_id}",
        "AssociatedTeamName": manager["TeamName"],
        "ManagerUserID": manager["UserID"]
    })
    for j in range(NUM_TASKS // NUM_MANAGERS):
        task_id = i * (NUM_TASKS // NUM_MANAGERS) + j + 1
        tasks.append({
            "TaskID": task_id,
            "TaskName": f"Task {task_id}",
            "Description": fake.text(),
            "Status": random.choice(["todo", "in progress", "done"]),
            "DueDate": fake.date_between(start_date="today", end_date="+30d"),
            "CompletionPercentage": random.randint(0, 100),
            "GoalID": goal_id
        })

# 5. Generate Feedbacks
feedbacks = []
for i in range(NUM_FEEDBACKS):
    user = random.choice(users[NUM_MANAGERS:])  # Employees only
    feedbacks.append({
        "FeedbackID": i + 1,
        "UserID": user["UserID"],
        "FeedbackContent": fake.sentence(),
        "Source": "Granite AI",
        "FeedbackTimestamp": fake.date_time_this_year()
    })

# 6. Generate Comments
comments = []
for i in range(NUM_COMMENTS):
    task = random.choice(tasks)
    user = random.choice(users)
    comments.append({
        "CommentID": i + 1,
        "TaskID": task["TaskID"],
        "UserID": user["UserID"],
        "CommentText": fake.sentence(),
        "CommentTimestamp": fake.date_time_this_year()
    })

# Convert to DataFrames
df_users = pd.DataFrame(users)
df_skills = pd.DataFrame(skills)
df_user_skills = pd.DataFrame(user_skills)
df_goals = pd.DataFrame(goals)
df_tasks = pd.DataFrame(tasks)
df_feedbacks = pd.DataFrame(feedbacks)
df_comments = pd.DataFrame(comments)

# Export to Excel
output_path = "synthetic_rag_dataset.xlsx"
with pd.ExcelWriter(output_path) as writer:
    df_users.to_excel(writer, sheet_name="Users", index=False)
    df_skills.to_excel(writer, sheet_name="Skills", index=False)
    df_user_skills.to_excel(writer, sheet_name="UserSkills", index=False)
    df_goals.to_excel(writer, sheet_name="Goals", index=False)
    df_tasks.to_excel(writer, sheet_name="Tasks", index=False)
    df_feedbacks.to_excel(writer, sheet_name="Feedbacks", index=False)
    df_comments.to_excel(writer, sheet_name="Comments", index=False)

output_path


'synthetic_rag_dataset.xlsx'

In [12]:
df_skills

,SkillID,SkillName,Description
0,1,"Designer, graphic",Defense issue century feel those.
1,2,Mining engineer,Allow nice future likely never though.
2,3,Data scientist,Physical behavior inside set throughout.
3,4,Market researcher,Yard until leg too value bag.
4,5,Applications developer,Certainly mission indeed agreement base baby.


In [19]:
data=pd.read_excel('synthetic_rag_dataset.xlsx')

In [14]:
data

,UserID,FirstName,LastName,IsActive,Role,EmployeeNumber,ReportsToUserID,TeamID,TeamName
0,1,Robert,Crawford,True,manager,MGR001,NaN,TEAM1,Team Cyan
1,2,Maxwell,Johnson,True,manager,MGR002,NaN,TEAM2,Team Indigo
2,3,Kathryn,Dudley,True,manager,MGR003,NaN,TEAM3,Team Violet
3,4,Mary,Martin,True,employee,EMP001,2.0,TEAM2,Team Indigo
4,5,Ashley,Espinoza,True,employee,EMP002,2.0,TEAM2,Team Indigo
5,6,Curtis,Santos,True,employee,EMP003,3.0,TEAM3,Team Violet
6,7,Karen,Tran,True,employee,EMP004,2.0,TEAM2,Team Indigo
7,8,Cindy,Frost,True,employee,EMP005,1.0,TEAM1,Team Cyan
8,9,Rachael,Bautista,True,employee,EMP006,1.0,TEAM1,Team Cyan
9,10,Daniel,Stein,True,employee,EMP007,2.0,TEAM2,Team Indigo


In [18]:
import pandas as pd

In [22]:
# Charger toutes les feuilles du fichier Excel
excel_path = "synthetic_rag_dataset.xlsx"
sheets = pd.read_excel(excel_path, sheet_name=None)  # `None` lit toutes les feuilles

# Vérification des noms des feuilles
print(sheets.keys())


dict_keys(['Users', 'Skills', 'UserSkills', 'Goals', 'Tasks', 'Feedbacks', 'Comments'])


In [25]:
corpus = []

# --- Mapping Skills ---
skill_id_to_name = dict(zip(sheets['Skills']['SkillID'], sheets['Skills']['SkillName']))

# Mapping UserID → list of skill names
user_skills_map = {}
for _, row in sheets['UserSkills'].iterrows():
    skill_name = skill_id_to_name.get(row['SkillID'])
    if skill_name:
        user_skills_map.setdefault(row['UserID'], []).append(skill_name)

# --- Users with Skills ---
users_df = sheets['Users']
for _, row in users_df.iterrows():
    user_id = row['UserID']
    user_info = (
        f"Nom: {row['FirstName']} {row['LastName']}, "
        f"Rôle: {row['Role']}, "
        f"Équipe: {row['TeamName']}, "
        f"Statut: {'Actif' if row['IsActive'] else 'Inactif'}"
    )
    skills = user_skills_map.get(user_id, [])
    skill_str = ", ".join(skills) if skills else "Aucune compétence spécifiée"
    corpus.append(f"Utilisateur: {user_info}, Compétences: {skill_str}")

# --- Skills ---
for _, row in sheets['Skills'].iterrows():
    skill_info = f"Compétence: {row['SkillName']}, Description: {row['Description']}"
    corpus.append(f"Compétence: {skill_info}")

# --- Goals ---
for _, row in sheets['Goals'].iterrows():
    goal_info = f"Objectif: {row['GoalName']}, Équipe associée: {row['AssociatedTeamName']}, Manager: {row['ManagerUserID']}"
    corpus.append(f"Objectif: {goal_info}")

# --- Tasks ---
for _, row in sheets['Tasks'].iterrows():
    task_info = f"Tâche: {row['TaskName']}, Description: {row['Description']}, Statut: {row['Status']}, Échéance: {row['DueDate']}"
    corpus.append(f"Tâche: {task_info}")

# --- Feedbacks ---
for _, row in sheets['Feedbacks'].iterrows():
    feedback_info = f"Feedback pour {row['UserID']}: {row['FeedbackContent']}, Source: {row['Source']}, Date: {row['FeedbackTimestamp']}"
    corpus.append(f"Feedback: {feedback_info}")

# --- Comments ---
for _, row in sheets['Comments'].iterrows():
    comment_info = f"Commentaire sur la tâche {row['TaskID']} par l'utilisateur {row['UserID']}: {row['CommentText']}, Date: {row['CommentTimestamp']}"
    corpus.append(f"Commentaire: {comment_info}")

# --- Preview ---
print("\n".join(corpus[:5]))  # Show first 5 entries for verification


Utilisateur: Nom: Kimberly Haley, Rôle: manager, Équipe: Team LightGreen, Statut: Actif, Compétences: Energy engineer, Financial trader, Textile designer
Utilisateur: Nom: Julia Harris, Rôle: manager, Équipe: Team Black, Statut: Actif, Compétences: Energy engineer, Recycling officer, Engineer, civil (contracting)
Utilisateur: Nom: Breanna Fleming, Rôle: manager, Équipe: Team Red, Statut: Actif, Compétences: Recycling officer
Utilisateur: Nom: Michelle Reeves, Rôle: employee, Équipe: Team Black, Statut: Actif, Compétences: Energy engineer, Recycling officer, Financial trader
Utilisateur: Nom: George Barrett, Rôle: employee, Équipe: Team Red, Statut: Actif, Compétences: Recycling officer


verify corpus

In [28]:

print("\n".join(corpus[5:10]))  


print("\n".join(corpus[10:15]))  


print("\n".join(corpus[15:20]))  


print("\n".join(corpus[20:25])) 


print("\n".join(corpus[25:30]))  


Utilisateur: Nom: David Brown, Rôle: employee, Équipe: Team Red, Statut: Actif, Compétences: Financial trader
Utilisateur: Nom: Steve Barnett, Rôle: employee, Équipe: Team Black, Statut: Actif, Compétences: Engineer, civil (contracting)
Utilisateur: Nom: Brian Thomas, Rôle: employee, Équipe: Team LightGreen, Statut: Actif, Compétences: Engineer, civil (contracting)
Utilisateur: Nom: Jennifer Ortiz, Rôle: employee, Équipe: Team LightGreen, Statut: Actif, Compétences: Financial trader
Utilisateur: Nom: Brittany Bailey, Rôle: employee, Équipe: Team Red, Statut: Actif, Compétences: Engineer, civil (contracting), Financial trader, Recycling officer
Utilisateur: Nom: Adrian Stevens, Rôle: employee, Équipe: Team LightGreen, Statut: Actif, Compétences: Engineer, civil (contracting)
Utilisateur: Nom: Tyler Evans, Rôle: employee, Équipe: Team LightGreen, Statut: Actif, Compétences: Financial trader, Engineer, civil (contracting), Textile designer
Utilisateur: Nom: Emily Norman, Rôle: employee, É

convert the corpus to rag supported format

In [30]:
df_skills

,SkillID,SkillName,Description
0,1,Customer service manager,Happy realize after machine stay.
1,2,"Surveyor, rural practice",Style available agency series.
2,3,Ecologist,Key land agent safe move true.
3,4,"Programme researcher, broadcasting/film/video",Agent mean room major build white something.
4,5,Market researcher,Side purpose evening.


In [31]:
corpus = []

# --- Create mappings ---
# SkillID → SkillName
skill_id_to_name = dict(zip(df_skills['SkillID'], df_skills['SkillName']))

# UserID → List of SkillNames (via df_user_skills)
user_skills_map = {}
for _, row in df_user_skills.iterrows():
    user_id = row['UserID']
    skill_id = row['SkillID']
    skill_name = skill_id_to_name.get(skill_id)
    if skill_name:
        user_skills_map.setdefault(user_id, []).append(skill_name)

# --- Add Users ---
for _, row in df_users.iterrows():
    user_id = row['UserID']
    full_name = f"{row['FirstName']} {row['LastName']}"
    role = row['Role']
    team = row['TeamName']
    status = "Actif" if row['IsActive'] else "Inactif"
    skills = user_skills_map.get(user_id, [])
    skills_text = ", ".join(skills) if skills else "Aucune"

    user_description = f"Nom: {full_name}, Rôle: {role}, Équipe: {team}, Statut: {status}, Compétences: {skills_text}"

    corpus.append({
        "document": user_description,
        "id": f"user_{user_id}",
        "metadata": {"type": "employee" if role.lower() == "employee" else "manager"}
    })

# --- Add Skills ---
for _, row in df_skills.iterrows():
    corpus.append({
        "document": f"Compétence: {row['SkillName']}, Description: {row['Description']}",
        "id": f"skill_{row['SkillID']}",
        "metadata": {"type": "skill"}
    })

# --- Add Goals ---
for _, row in df_goals.iterrows():
    corpus.append({
        "document": (
            f"Objectif: {row['GoalName']}, "
            f"Équipe associée: {row['AssociatedTeamName']}, "
            f"Manager: {row['ManagerUserID']}"
        ),
        "id": f"goal_{row['GoalID']}",
        "metadata": {"type": "goal"}
    })

# --- Add Tasks ---
for _, row in df_tasks.iterrows():
    corpus.append({
        "document": (
            f"Tâche: {row['TaskName']}, "
            f"Description: {row['Description']}, "
            f"Statut: {row['Status']}, "
            f"Échéance: {row['DueDate']}"
        ),
        "id": f"task_{row['TaskID']}",
        "metadata": {"type": "task"}
    })

# --- Add Feedbacks ---
for _, row in df_feedbacks.iterrows():
    corpus.append({
        "document": (
            f"Feedback pour l'utilisateur {row['UserID']}: {row['FeedbackContent']}, "
            f"Source: {row['Source']}, "
            f"Date: {row['FeedbackTimestamp']}"
        ),
        "id": f"feedback_{row['FeedbackID']}",
        "metadata": {"type": "feedback"}
    })

# --- Add Comments ---
for _, row in df_comments.iterrows():
    corpus.append({
        "document": (
            f"Commentaire sur la tâche {row['TaskID']} par l'utilisateur {row['UserID']}: "
            f"{row['CommentText']}, "
            f"Date: {row['CommentTimestamp']}"
        ),
        "id": f"comment_{row['CommentID']}",
        "metadata": {"type": "comment"}
    })

# --- Preview ---
print(corpus[:10])


[{'document': 'Nom: Kimberly Haley, Rôle: manager, Équipe: Team LightGreen, Statut: Actif, Compétences: Energy engineer, Financial trader, Textile designer', 'id': 'user_1', 'metadata': {'type': 'manager'}}, {'document': 'Nom: Julia Harris, Rôle: manager, Équipe: Team Black, Statut: Actif, Compétences: Energy engineer, Recycling officer, Engineer, civil (contracting)', 'id': 'user_2', 'metadata': {'type': 'manager'}}, {'document': 'Nom: Breanna Fleming, Rôle: manager, Équipe: Team Red, Statut: Actif, Compétences: Recycling officer', 'id': 'user_3', 'metadata': {'type': 'manager'}}, {'document': 'Nom: Michelle Reeves, Rôle: employee, Équipe: Team Black, Statut: Actif, Compétences: Energy engineer, Recycling officer, Financial trader', 'id': 'user_4', 'metadata': {'type': 'employee'}}, {'document': 'Nom: George Barrett, Rôle: employee, Équipe: Team Red, Statut: Actif, Compétences: Recycling officer', 'id': 'user_5', 'metadata': {'type': 'employee'}}, {'document': 'Nom: David Brown, Rôle:

Generate embeddings

In [34]:
corpus

[{'document': 'Nom: Kimberly Haley, Rôle: manager, Équipe: Team LightGreen, Statut: Actif, Compétences: Energy engineer, Financial trader, Textile designer',
  'id': 'user_1',
  'metadata': {'type': 'manager'}},
 {'document': 'Nom: Julia Harris, Rôle: manager, Équipe: Team Black, Statut: Actif, Compétences: Energy engineer, Recycling officer, Engineer, civil (contracting)',
  'id': 'user_2',
  'metadata': {'type': 'manager'}},
 {'document': 'Nom: Breanna Fleming, Rôle: manager, Équipe: Team Red, Statut: Actif, Compétences: Recycling officer',
  'id': 'user_3',
  'metadata': {'type': 'manager'}},
 {'document': 'Nom: Michelle Reeves, Rôle: employee, Équipe: Team Black, Statut: Actif, Compétences: Energy engineer, Recycling officer, Financial trader',
  'id': 'user_4',
  'metadata': {'type': 'employee'}},
 {'document': 'Nom: George Barrett, Rôle: employee, Équipe: Team Red, Statut: Actif, Compétences: Recycling officer',
  'id': 'user_5',
  'metadata': {'type': 'employee'}},
 {'document':

In [37]:
#!pip install torch==1.12.1 --force-reinstall

import torch
from sentence_transformers import SentenceTransformer
embedding_model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
embeddings = []
for doc in corpus:
    text = doc["document"]
    with torch.no_grad():  
        embedding = embedding_model.encode(text)
    embeddings.append({"id": doc["id"], "embedding": embedding, "document": text})

print(embeddings[:10])  

[{'id': 'user_1', 'embedding': array([-3.67856584e-02, -5.17001562e-03,  4.33102809e-03,  4.79755066e-02,
        2.61155032e-02,  1.05931321e-02,  7.01401904e-02, -4.66421572e-03,
       -3.22595127e-02,  1.86264992e-03,  3.61623690e-02, -1.37988999e-01,
        7.25948135e-04, -3.50189507e-02, -1.39917422e-03,  5.86201400e-02,
        1.49357133e-02,  2.24956777e-02, -6.49893703e-03, -8.96009058e-02,
       -7.57896230e-02, -8.47557858e-02,  7.36288503e-02, -1.08916790e-03,
       -2.32169293e-02, -3.21173705e-02,  2.54663522e-04,  2.15825401e-02,
       -3.53000946e-02, -1.63177982e-01,  1.35335140e-02,  1.15968600e-01,
        2.09653703e-03,  1.02284789e-01,  1.42789871e-01,  1.48008257e-01,
       -4.75455867e-03, -1.31591195e-02, -8.42739418e-02, -1.29890386e-02,
       -2.28643809e-02, -9.28429216e-02, -6.21920153e-02, -6.78369403e-02,
       -3.61245871e-03, -3.26207392e-02,  7.86071494e-02, -5.97041920e-02,
       -4.98632342e-02, -1.78497080e-02, -6.59965947e-02, -2.18854565

In [43]:
import faiss
import numpy as np



vectors = np.array([e['embedding'] for e in embeddings]).astype("float32")
ids = [e['id'] for e in embeddings]
dimension = vectors.shape[1]
index = faiss.IndexFlatL2(dimension)
index.add(vectors)


In [46]:
index

<faiss.swigfaiss_avx2.IndexFlatL2; proxy of <Swig Object of type 'faiss::IndexFlatL2 *' at 0x7fc7432cadf0> >

In [49]:
query = "Manager of Team DarkMagenta"
query_embedding = embedding_model.encode([query]).astype("float32")
k = 3
distances, indices = index.search(query_embedding, k)
for i, idx in enumerate(indices[0]):
    print(f"{i+1}. ID: {ids[idx]}, Distance: {distances[0][i]}")


1. ID: goal_1, Distance: 1.0328782796859741
2. ID: goal_3, Distance: 1.1244169473648071
3. ID: goal_2, Distance: 1.1511917114257812


In [52]:
embeddings

[{'id': 'user_1',
  'embedding': array([-3.67856584e-02, -5.17001562e-03,  4.33102809e-03,  4.79755066e-02,
          2.61155032e-02,  1.05931321e-02,  7.01401904e-02, -4.66421572e-03,
         -3.22595127e-02,  1.86264992e-03,  3.61623690e-02, -1.37988999e-01,
          7.25948135e-04, -3.50189507e-02, -1.39917422e-03,  5.86201400e-02,
          1.49357133e-02,  2.24956777e-02, -6.49893703e-03, -8.96009058e-02,
         -7.57896230e-02, -8.47557858e-02,  7.36288503e-02, -1.08916790e-03,
         -2.32169293e-02, -3.21173705e-02,  2.54663522e-04,  2.15825401e-02,
         -3.53000946e-02, -1.63177982e-01,  1.35335140e-02,  1.15968600e-01,
          2.09653703e-03,  1.02284789e-01,  1.42789871e-01,  1.48008257e-01,
         -4.75455867e-03, -1.31591195e-02, -8.42739418e-02, -1.29890386e-02,
         -2.28643809e-02, -9.28429216e-02, -6.21920153e-02, -6.78369403e-02,
         -3.61245871e-03, -3.26207392e-02,  7.86071494e-02, -5.97041920e-02,
         -4.98632342e-02, -1.78497080e-02, -6

Perform search similarity

In [55]:

id_to_doc = {e['id']: e['document'] for e in embeddings}


for i, idx in enumerate(indices[0]):
    doc_id = ids[idx]
    print(f"{i+1}. ID: {doc_id}, Distance: {distances[0][i]}")
    print(f"   Document: {id_to_doc[doc_id]}\n")


1. ID: goal_1, Distance: 1.0328782796859741
   Document: Objectif: Goal 1, Équipe associée: Team LightGreen, Manager: 1

2. ID: goal_3, Distance: 1.1244169473648071
   Document: Objectif: Goal 3, Équipe associée: Team Red, Manager: 3

3. ID: goal_2, Distance: 1.1511917114257812
   Document: Objectif: Goal 2, Équipe associée: Team Black, Manager: 2



creating langchain docs

In [60]:
from langchain.schema import Document

langchain_docs = [
    Document(page_content=doc["document"], metadata={"id": doc["id"]})
    for doc in corpus
]



In [61]:
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings

embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
vectorstore = FAISS.from_documents(langchain_docs, embedding_model)


/tmp/ipykernel_172/777240507.py:5: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")


In [64]:
vectorstore

In [70]:
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
retriever = vectorstore.as_retriever()

In [67]:
query = "Manager of Team darkagentaTan"
results = vectorstore.similarity_search(query, k=3)

for i, doc in enumerate(results):
    print(f"{i+1}. ID: {doc.metadata['id']}")
    print(f"   Document: {doc.page_content}\n")


1. ID: goal_1
   Document: Objectif: Goal 1, Équipe associée: Team LightGreen, Manager: 1

2. ID: goal_3
   Document: Objectif: Goal 3, Équipe associée: Team Red, Manager: 3

3. ID: user_3
   Document: Nom: Breanna Fleming, Rôle: manager, Équipe: Team Red, Statut: Actif, Compétences: Recycling officer



separate between employee and taks 

In [77]:
from langchain.schema import Document

langchain_docs = []
for doc in corpus:
    if "Nom:" in doc["document"] and "Rôle:" in doc["document"]:
        doc_type = "employee"
    else:
        doc_type = "task"

    langchain_docs.append(
        Document(
            page_content=doc["document"],
            metadata={"id": doc["id"], "type": doc_type}
        )
    )


In [80]:
vectorstore = FAISS.from_documents(langchain_docs, embedding_model)
retriever = vectorstore.as_retriever()


First Model call

In [86]:
# 1. PROPER IMPORTS
from langchain_community.llms import WatsonxLLM
from ibm_watson_machine_learning.metanames import GenTextParamsMetaNames as GenParams
from langchain.chains import RetrievalQA

# 2. FIXED COMPATIBILITY LAYER
class WatsonxLLMCompat(WatsonxLLM):
    def _call(self, prompt, **kwargs):
        # Convert LangChain parameters to IBM format
        watson_params = {
            GenParams.MAX_NEW_TOKENS: 300,
            GenParams.MIN_NEW_TOKENS: 1,
            GenParams.DECODING_METHOD: "greedy"
        }
        watson_params.update(kwargs.pop("parameters", {}))
        
        return super()._call(
            prompt=prompt,
            params=watson_params,
            **kwargs
        )

# 3. INITIALIZE WITH CREDENTIALS (from your existing model config)
watsonx_llm = WatsonxLLMCompat(
    model_id="ibm/granite-13b-instruct-v2",
    url="https://us-south.ml.cloud.ibm.com",
    apikey=os.getenv("WATSONX_API_KEY"),
    project_id=os.getenv("WATSONX_PROJECT_ID")
)

# 4. RAG CHAIN SETUP
rag_chain = RetrievalQA.from_chain_type(
    llm=watsonx_llm,
    retriever=retriever,
    return_source_documents=True
)

query = "hi, how are you"
result = rag_chain.invoke({"query": query})
print("Réponse :")
print(result["result"])
print("\nDocuments sources :")
for doc in result["source_documents"]:
    print("-", doc.page_content)


Réponse :
 Hello, I am well. Thank you. How are you? I am doing fine. Thank you

Documents sources :
- Nom: Emily Norman, Rôle: employee, Équipe: Team Red, Statut: Actif, Compétences: Engineer, civil (contracting), Textile designer
- Nom: Breanna Fleming, Rôle: manager, Équipe: Team Red, Statut: Actif, Compétences: Recycling officer
- Feedback pour l'utilisateur 8: Fall mention computer., Source: Granite AI, Date: 2025-05-02 07:55:37.253777
- Feedback pour l'utilisateur 5: Approach form state adult most respond., Source: Granite AI, Date: 2025-04-17 22:25:53.134173


First try

In [188]:
def assign_task_with_rag(task_details):
    # Get relevant documents from retriever
    retrieved_docs = retriever.get_relevant_documents(
        f"Task: {task_details['title']}, Skills: {', '.join(task_details['required_skills'])}"
    )

    # ⚠️ Filter only employee-related documents
    employee_docs = [doc for doc in retrieved_docs if doc.metadata.get("type") == "employee"]

    # If no employee docs found, exit early
    if not employee_docs:
        print("No relevant employee documents found.")
        return

    # Build the context string from employee documents
    context = "\n".join([doc.page_content for doc in employee_docs])

    # Refined prompt with more detailed context
    prompt = f"""
You are an intelligent assistant helping assign tasks to the most suitable employee based on internal company documents, including team membership, roles, goals, competencies, and feedback.

### Task to assign:
- Title: {task_details['title']}
- Required Skills: {', '.join(task_details['required_skills'])}
- Priority: {task_details['priority']}
- Deadline: {task_details['deadline']}

### Contextual documents:
{context}

### Instruction:
From the context provided above, choose the most suitable **employee** for this task based on:
- Skill match
- Team alignment with objectives
- Task availability and current workload
- Manager relationships if applicable

Please respond with:
1. A short explanation (2–3 sentences) justifying your choice.
2. Then write: ` Employee ID`
"""

    # Query the RAG chain
    result = rag_chain.invoke({"query": prompt})

    # Display the generated result and documents
    print("Assigned Employee ID:")
    print(result["result"])

    print("\nDocuments used for task assignment:")
    for doc in employee_docs:
        print("-", doc.page_content)


In [191]:
# Test the function
task = {
    "title": "Design and launch marketing campaign for new eco-friendly product line",
    "required_skills": ["Arts development officer", "Designer, jewellery"],
    "priority": "Medium",
    "deadline": "2025-06-15"
}

assign_task_with_rag(task)

Assigned Employee ID:

1. Because the employee has the required skills and experience.
2. Tyler Evans

Documents used for task assignment:
- Nom: Emily Norman, Rôle: employee, Équipe: Team Red, Statut: Actif, Compétences: Engineer, civil (contracting), Textile designer
- Nom: Tyler Evans, Rôle: employee, Équipe: Team LightGreen, Statut: Actif, Compétences: Financial trader, Engineer, civil (contracting), Textile designer
- Nom: Kimberly Haley, Rôle: manager, Équipe: Team LightGreen, Statut: Actif, Compétences: Energy engineer, Financial trader, Textile designer


In [167]:
def assign_task_with_rag(task_details):
    # Get relevant documents from retriever
    query = f"Task: {task_details['title']}, Skills: {', '.join(task_details['required_skills'])}"
    retrieved_docs = retriever.get_relevant_documents(query)

    # Filter only employee-related documents
    employee_docs = [doc for doc in retrieved_docs if doc.metadata.get("type") == "employee"]

    # If no employee docs found, exit early
    if not employee_docs:
        print("No relevant employee documents found.")
        return

    # Build the context string from employee documents
    employee_context = "\n".join([doc.page_content for doc in employee_docs])

    # Refined prompt with more detailed context
    prompt = f"""
You are an intelligent assistant helping assign tasks to the most suitable employee based on internal company documents, including team membership, roles, goals, competencies, and feedback.

### Task to assign:
- Title: {task_details['title']}
- Required Skills: {', '.join(task_details['required_skills'])}
- Priority: {task_details['priority']}
- Deadline: {task_details['deadline']}

### Contextual documents:
{employee_context}

### Instruction:
From the context provided above, choose the most suitable **employee** for this task based on:
- Skill match
- Team alignment with objectives
- Task availability and current workload
- Manager relationships if applicable

Please respond with:
1. A short explanation (2–3 sentences) justifying your choice.
2. Then write: `Assigned Employee ID: employeeX`
"""

    # Query the RAG chain
    result = rag_chain.invoke({"query": prompt})

    # Display the generated result and documents
    print("Assignment Result:")
    print(result["result"])

    print("\nDocuments used for task assignment:")
    for doc in employee_docs:
        print("-", doc.page_content)

# Test the function
task = {
    "title": "Design and launch marketing campaign for new eco-friendly product line",
    "required_skills": ["Arts development officer", "Designer, jewellery"],
    "priority": "Medium",
    "deadline": "2025-06-15"
}

assign_task_with_rag(task)

Assignment Result:

1. Because the employee has the required skills and experience.
2. employeeX

Documents used for task assignment:
- Nom: Emily Norman, Rôle: employee, Équipe: Team Red, Statut: Actif, Compétences: Engineer, civil (contracting), Textile designer
- Nom: Tyler Evans, Rôle: employee, Équipe: Team LightGreen, Statut: Actif, Compétences: Financial trader, Engineer, civil (contracting), Textile designer
- Nom: Kimberly Haley, Rôle: manager, Équipe: Team LightGreen, Statut: Actif, Compétences: Energy engineer, Financial trader, Textile designer


Enhance the rag prompt

In [173]:
def assign_task_with_rag(task_details):
    # Get relevant documents from retriever
    query = f"Task: {task_details['title']}, Skills: {', '.join(task_details['required_skills'])}"
    retrieved_docs = retriever.get_relevant_documents(query)

    # Filter only employee-related documents
    employee_docs = [doc for doc in retrieved_docs if doc.metadata.get("type") == "employee"]

    # If no employee docs found, exit early
    if not employee_docs:
        print("No relevant employee documents found.")
        return

    # Build the context string from employee documents
    context = "\n".join([doc.page_content for doc in employee_docs])

    prompt = f"""### Task Assignment Analysis
**Task Requirements**:
- Title: {task_details['title']}
- Required Skills: {', '.join(task_details['required_skills'])}
- Priority: {task_details['priority']}
- Deadline: {task_details['deadline']}

**Candidate Profiles**:
{context}

**Instructions**:
1. Analyze skill matches between task requirements and employee competencies
2. Consider team alignment with current objectives
3. Evaluate workload indicators from status and recent assignments
4. Select the best candidate using format:
   **Chosen Employee**: [Name]
   **Reason**: [2-3 sentence justification]
   **Employee ID**: [ID from documents]

**Response must complete ALL sections above**"""

    # Force structured output
    result = rag_chain.invoke({
        "query": prompt,
        "parameters": {"stop": ["\n\n", "###"]}  # Hard stop triggers
    })
    print("Assignment Result:")
    print(result["result"])

    print("\nDocuments used for task assignment:")
    for doc in employee_docs:
        print("-", doc.page_content)

# Test the function
task = {
    "title": "Design and launch marketing campaign for new eco-friendly product line",
    "required_skills": ["Financial trader", "designer, jewellery"],
    "priority": "Medium",
    "deadline": "2025-06-15"
}
assign_task_with_rag(task)

Assignment Result:

Chosen Employee: Tyler Evans
Reason: He has all the skills required for this task.

Documents used for task assignment:
- Nom: Tyler Evans, Rôle: employee, Équipe: Team LightGreen, Statut: Actif, Compétences: Financial trader, Engineer, civil (contracting), Textile designer
- Nom: Emily Norman, Rôle: employee, Équipe: Team Red, Statut: Actif, Compétences: Engineer, civil (contracting), Textile designer
- Nom: David Brown, Rôle: employee, Équipe: Team Red, Statut: Actif, Compétences: Financial trader


In [122]:
%who


APIClient	 Dict	 Document	 FAISS	 Faker	 GenParams	 HuggingFaceEmbeddings	 List	 MAX_SKILLS_PER_USER	 
Model	 NUM_COMMENTS	 NUM_EMPLOYEES	 NUM_FEEDBACKS	 NUM_MANAGERS	 NUM_SKILLS	 NUM_TASKS	 Optional	 RetrievalQA	 
SentenceTransformer	 WatsonxLLM	 WatsonxLLMCompat	 assign_task	 assign_task_with_rag	 client	 comment_info	 comments	 corpus	 
credentials	 data	 datetime	 df_comments	 df_feedbacks	 df_goals	 df_skills	 df_tasks	 df_user_skills	 
df_users	 dimension	 distances	 doc	 doc_id	 doc_type	 embedding	 embedding_model	 embeddings	 
excel_path	 faiss	 fake	 feedback_info	 feedbacks	 full_name	 goal_id	 goal_info	 goals	 
i	 id_to_doc	 ids	 idx	 index	 indices	 j	 k	 langchain_docs	 
load_dotenv	 manager	 model	 model_id	 np	 os	 output_path	 pd	 project_id	 
query	 query_embedding	 rag_chain	 random	 result	 results	 retriever	 role	 row	 
sheets	 skill_id	 skill_id_to_name	 skill_ids	 skill_info	 skill_name	 skill_str	 skills	 skills_text	 
status	 task	 task_id	 task_info	 tasks	 

Combine the generation process

In [137]:
import joblib
# Save FAISS index using joblib
joblib.dump(vectorstore, 'faiss_index.joblib')

['faiss_index.joblib']

In [140]:
retriever = vectorstore.as_retriever()

In [143]:
rag_chain = RetrievalQA.from_chain_type(
    llm=watsonx_llm,
    retriever=retriever,
    return_source_documents=True
)

In [158]:
!pip freeze > requirements.txt


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [155]:
from typing import Dict, List, Optional

def assign_task(
    retriever: object,  # Should be your FAISS retriever instance
    rag_chain: object,  # Your initialized RetrievalQA chain
    task_details: Dict[str, str],
    max_docs: int = 5,
    temperature: float = 0.3,
    max_tokens: int = 600
) -> Optional[Dict]:
    """
    Assigns tasks to employees using RAG-powered analysis.
    
    Args:
        retriever: Document retriever instance
        rag_chain: Initialized RAG chain
        task_details: Dictionary containing:
            - title: Task name
            - required_skills: List of required skills
            - priority: Task priority level
            - deadline: Due date
        max_docs: Maximum documents to consider
        temperature: Generation creativity (0-1)
        max_tokens: Maximum response length

    Returns:
        Dict with assignment result and source documents
    """
    # Document retrieval and filtering
    query = f"Task: {task_details['title']}, Skills: {', '.join(task_details['required_skills'])}"
    retrieved_docs = retriever.get_relevant_documents(query)[:max_docs]
    employee_docs = [doc for doc in retrieved_docs if doc.metadata.get("type") == "employee"]

    if not employee_docs:
        print("No relevant employee documents found.")
        return None

    # Build context and generate prompt
    context = "\n".join(doc.page_content for doc in employee_docs)
    prompt = _build_assignment_prompt(task_details, context)

    # Generate response with controlled parameters
    result = rag_chain.invoke({
        "query": prompt,
        "parameters": {
            "temperature": temperature,
            "max_tokens": max_tokens,
            "stop": ["\n\n", "###"]
        }
    })

    return {
        "result": result["result"],
        "documents": [doc.page_content for doc in employee_docs]
    }

def _build_assignment_prompt(task_details: Dict, context: str) -> str:
    """Constructs structured prompt for task assignment."""
    return f"""### Task Assignment Analysis
**Task Requirements**:
- Title: {task_details['title']}
- Required Skills: {', '.join(task_details['required_skills'])}
- Priority: {task_details['priority']}
- Deadline: {task_details['deadline']}

**Candidate Profiles**:
{context}

**Instructions**:
1. Match required skills with employee competencies
2. Consider team alignment with current objectives
3. Evaluate workload indicators (status/activity)
4. Format response as:
   **Chosen Employee**: [Full Name]
   **Reason**: [Concise 2-3 sentence justification]
   **Employee ID**: [Document Reference ID]

**Response must include all three sections**"""

# Usage Example
if __name__ == "__main__":
    task = {
        "title": "Design eco-friendly product campaign",
        "required_skills": ["Arts development", "Jewellery design"],
        "priority": "Medium",
        "deadline": "2025-06-15"
    }

    assignment = assign_task(
        retriever=retriever,
        rag_chain=rag_chain,
        task_details=task
    )

    if assignment:
        print("Assignment Result:\n", assignment["result"])
        print("\nDocuments Used:")
        for doc in assignment["documents"]:
            print(f"- {doc}")


Assignment Result:
 
1. Nom: Emily Norman, Rôle: employee, Équipe: Team Red,

Documents Used:
- Nom: Emily Norman, Rôle: employee, Équipe: Team Red, Statut: Actif, Compétences: Engineer, civil (contracting), Textile designer
- Nom: Tyler Evans, Rôle: employee, Équipe: Team LightGreen, Statut: Actif, Compétences: Financial trader, Engineer, civil (contracting), Textile designer
- Nom: Breanna Fleming, Rôle: manager, Équipe: Team Red, Statut: Actif, Compétences: Recycling officer


In [152]:
result = assign_task(retriever, rag_chain, task_details)
if result:
    process_result(result["result"])
    analyze_documents(result["documents"])


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=d7f656e2-1e80-46d1-8b70-ddea07451e8b' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>